In [1]:
#2025-04-28-tutorial

from IPython.display import display, Math


In [2]:
display(Math(r' H =  -J_1 \sum_{i=1}^{L-1} \sigma_i^z \sigma_{i+1}^z  - J_2 \sum_{i=1}^{L-2} \sigma_i^z \sigma_{i+2}^z  - h \sum_{i=1}^{L} \sigma_i^x  - g \sum_{i=1}^{L} \sigma_i^z  - \frac{J_{xy}}{2} (1 + \gamma) \sum_{i=1}^{L-1} (1 - (-1)^i \delta)\, \sigma_i^x \sigma_{i+1}^x  - \frac{J_{xy}}{2} (1 - \gamma) \sum_{i=1}^{L-1} (1 - (-1)^i \delta)\, \sigma_i^y \sigma_{i+1}^y'))

<IPython.core.display.Math object>

In [3]:
display(Math(r'\text{Tutorial 2: energy gap and trace distance as SDPs}'))

<IPython.core.display.Math object>

In [4]:
display(Math(r'\text{Quantum Ising model (with open boundary conditions)}'))
display(Math(r'H = -J_1 \sum_{\langle i,j \rangle} \sigma_i^z \sigma_j^z - h \sum_i \sigma_i^x'))
display(Math(r'\text{Define Hamiltonian and set the parameters}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import kron, identity, csr_matrix
from scipy.sparse.linalg import eigsh
 

# Define Pauli matrices
sx = csr_matrix(np.array([[0, 1], [1, 0]], dtype=complex))
sy = csr_matrix(np.array([[0, -1j], [1j, 0]], dtype=complex))
sz = csr_matrix(np.array([[1, 0], [0, -1]], dtype=complex))
id2 = identity(2, format='csr', dtype=complex)

def kron_n(ops):
    """Kronecker product of a list of operators."""
    result = ops[0]
    for op in ops[1:]:
        result = kron(result, op, format='csr')
    return result

def build_hamiltonian(L, J1, J2, h, g, J_xy, delta, gamma):
    H = csr_matrix((2**L, 2**L), dtype=complex)

    # Build -J1 * sum sigma^z_i sigma^z_{i+1}
    for i in range(L-1):
        ops = [id2] * L
        ops[i] = sz
        ops[(i + 1)] = sz
        H -= J1 * kron_n(ops)

    # Build -J2 * sum sigma^z_i sigma^z_{i+2}
    for i in range(L-2):
        ops = [id2] * L
        ops[i] = sz
        ops[(i + 2)] = sz
        H -= J2 * kron_n(ops)

    # Build -h * sum sigma^x_i
    for i in range(L):
        ops = [id2] * L
        ops[i] = sx
        H -= h * kron_n(ops)


    # build -g * sum sigma^z_i 
    for i in range(L):
        ops = [id2] * L
        ops[i] = sz
        H -= g * kron_n(ops)

    # build dimerized hamiltonian
    for i in range(L-1):
        ops = [id2] * L
        ops[i] = sx
        ops[(i + 1)] = sx
        H -= J_xy*(1-(-1)**i*delta)*(1+gamma)/2 * kron_n(ops)

    for i in range(L-1):
        ops = [id2] * L
        ops[i] = sy
        ops[(i + 1)] = sy
        H -= J_xy*(1-(-1)**i*delta)*(1-gamma)/2 * kron_n(ops)

    
    #return the Hamiltonian
    return H


L=4; 
 
J1=1; h=0.3;  
J_xy=0; delta=0.; gamma=0.; J2=0; g=0.; 

H=build_hamiltonian(L, J1, J2, h, g, J_xy, delta, gamma)

# Diagonalize the Hamiltonian
eigenvalues, eigenvectors = eigsh(H, k=2, which='SA')  # Calculate the the gnd state and the first excited state
#‘SA’ : Smallest (algebraic) eigenvalues.
 

print(f" H0 has L={L}, J1={J1},J2={J2}, h={h}, g={g}, J_xy={J_xy}, delta={delta}, gamma={gamma} ")
print("the gnd state energy is "+ str(np.min(eigenvalues)))
print("the first excited energy is "+ str(np.max(eigenvalues)))
gap_diag=np.max(eigenvalues)-np.min(eigenvalues)


 
print("the gap is "+ str(gap_diag))



 
Egs=np.min(eigenvalues)




 H0 has L=4, J1=1,J2=0, h=0.3, g=0.0, J_xy=0, delta=0.0, gamma=0.0 
the gnd state energy is -3.1433267964028113
the first excited energy is -3.128581360641489
the gap is 0.014745435761322145


In [6]:
display(Math(r'\text{Find the gap with an SDP}'))
display(Math(r'\text{see, e.g., arXiv:2411.03680}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
#gap via SDP

import cvxpy as cp

Hresc=H-Egs*identity(2**L, format='csr', dtype=complex)

#just a check that the min eigenvalue is now zero
# Diagonalize the Hamiltonian
eigenvalues0, eigenvectors = eigsh(Hresc, k=1, which='SA')  # Calculate ground state

print(f" H has L={L}, J1={J1},J2={J2}, h={h}, g={g}, J_xy={J_xy}, delta={delta}, gamma={gamma} ")
print("the min eigval has to be zero now: "+str(eigenvalues0[0]))


#define sdp viariables
X = cp.Variable((1, 1), hermitian=True)  #the quantum state
#define constriants for the quantum state
constraints = []
constraints += [Hresc@Hresc-X*Hresc >> 0]

#define the problem
prob = cp.Problem(cp.Maximize(cp.real(cp.trace(X))), constraints)
prob.solve(solver=cp.MOSEK, verbose=False)#set verbose to "True" to see details on the calculation

print("problem status: ",prob.status)
print("The gap computed with the sdp is", prob.value)

 H has L=4, J1=1,J2=0, h=0.3, g=0.0, J_xy=0, delta=0.0, gamma=0.0 
the min eigval has to be zero now: -4.64606548714879e-16
problem status:  optimal
The gap computed with the sdp is 0.014745435736031754


In [8]:
display(Math(r'\text{Trace distance of two states as a SDP}'))








<IPython.core.display.Math object>

In [9]:
#define two random states rho and sigma

def random_dm(n):
    A = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    H = (A + A.conj().T) / 2  # make it Hermitian
    H1=H.conj().T@H  #we make it positive
    H2=H1/np.trace(H1)
    return H2

# Example: 4x4 Hermitian matrix
#rho = random_dm(4)
#print(np.linalg.eigh(rho)[0])
#print(np.trace(rho))

d=4

rho=random_dm(d)
sigma=random_dm(d)

In [10]:
print("trace norm of (rho - sigma) is",np.linalg.norm(rho-sigma, ord='nuc'))  #trace norm of the difference

trace norm of (rho - sigma) is 1.1072486590000032


In [11]:

#define sdp viariables
E1 = cp.Variable((d, d), hermitian=True)  #first POVM element
E2 = cp.Variable((d, d), hermitian=True)  #second POVM element


#define constriants for the quantum state
constraints = []
constraints += [E1 >> 0]
constraints += [E2 >> 0]

constraints += [E1+E2 == np.eye(d)]

#define the problem
prob = cp.Problem(cp.Maximize(cp.real(cp.trace(E1@(rho-sigma))+cp.trace(E2@(sigma-rho)))), constraints)
prob.solve(solver=cp.MOSEK, verbose=False)#set verbose to true to see details on the calculation

print("problem status: ",prob.status)
print("SDP calculation: trace norm of (rho - sigma) is", prob.value)

E1best=E1.value
E2best=E2.value

print("we can check the optimal measurement is projective")
print("norm(E1^2-E1)=",np.linalg.norm(E1best@E1best-E1best, ord='fro'))



problem status:  optimal
SDP calculation: trace norm of (rho - sigma) is 1.107248649397817
we can check the optimal measurement is projective
norm(E1^2-E1)= 1.5877063145561203e-08
